In [1]:
import numpy as np
import gurobipy as gp

In [2]:
count_units = 3
count_times = 6

costs_startup_const = [800, 500, 250]
costs_generation = [5, 15, 30]
power_outputs_minimum = [80, 50, 30]
power_outputs_maximum = [300, 200, 100]
startups = [100, 70, 40]
startdowns = [80, 50, 30]
rampups = [50, 60, 70]
rampdowns = [30, 40, 50]
minimumups = [3, 2, 1]
minimumdowns = [2, 2, 2]
commitments_initial = [1, 0, 0]
power_outputs_initial = [120, 0, 0]
minimumups_initial  = [2, 0, 0]
minimumdowns_initial  = [0, 0, 0]
demands = [220, 250, 200, 170, 230, 190]
reserves = [10, 10, 10, 10, 10, 10]

# helper parameter
power_outputs_pseudo_ub = np.tile(np.array(power_outputs_maximum)[:, None], (1, count_times))

In [3]:
model = gp.Model()
model.setParam("OutputFlag", 0)

Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [4]:
# variable
commitments = model.addVars(range(count_units), range(count_times), vtype=gp.GRB.BINARY)
power_outputs = model.addVars(range(count_units), range(count_times), lb=0, ub=power_outputs_pseudo_ub)
power_outputs_bars = model.addVars(range(count_units), range(count_times), lb=0, ub=power_outputs_pseudo_ub)
costs_startup = model.addVars(range(count_units), range(count_times), lb=0)

In [5]:
# constr - balance between sum of power and demand at each time
model.addConstrs(
    gp.quicksum(power_outputs[i, t] for i in range(count_units)) == demands[t]
    for t in range(count_times)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [6]:
# constr - sum of power bar should be greater than demand and reserve at each time
model.addConstrs(
    gp.quicksum(power_outputs_bars[i, t] for i in range(count_units)) >= demands[t] + reserves[t]
    for t in range(count_times)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [7]:
# constraint - power min bound with commitment decision
model.addConstrs(
    power_outputs[i, t] >= power_outputs_minimum[i] * commitments[i, t]
    for i in range(count_units) for t in range(count_times)
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>}

In [8]:
# constraint - power max bound with commitment decision
model.addConstrs(
    power_outputs[i, t] <= power_outputs_maximum[i] * commitments[i, t]
    for i in range(count_units) for t in range(count_times)
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>}

In [9]:
# const - power can't be great than power bar that satisfies reserve
model.addConstrs(
    power_outputs[i, t] <= power_outputs_bars[i, t]
    for i in range(count_units) for t in range(count_times)
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>}

In [10]:
# const - start-up costs
#t = 1, 2, ..., T -1 
model.addConstrs(
    costs_startup[i, t] >= costs_startup_const[i] * (commitments[i, t] - commitments[i, t - 1])
    for i in range(count_units) for t in range(1, count_times)
)
# t = 0
model.addConstrs(
    costs_startup[i, 0] >= costs_startup_const[i] * (commitments[i, 0] - commitments_initial[i])
    for i in range(count_units)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [11]:
# const - ramp-up and startup
# for t = 1, 2, ..., T-1
model.addConstrs(
    power_outputs_bars[i, t] <= (
        power_outputs[i, t-1]
        + rampups[i] * commitments[i, t-1]
        + startups[i] * (commitments[i, t] - commitments[i, t-1])
        + power_outputs_maximum[i] * (1 - commitments[i, t])
    )
    for i in range(count_units) for t in range(1, count_times)
)
# t = 0
model.addConstrs(
    power_outputs_bars[i, 0] <= (
        power_outputs_initial[i] 
        + rampups[i] * commitments_initial[i]
        + startups[i] * (commitments[i, 0] - commitments_initial[i])
        + power_outputs_maximum[i] * (1 - commitments[i, 0])
    )
    for i in range(count_units)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [12]:
# const - shutdown ramp
# for t = 1, 2, ..., T-2
model.addConstrs(
    power_outputs_bars[i, t] <= (
        power_outputs_maximum[i] * commitments[i, t+1]
        + startdowns[i] * (commitments[i, t] - commitments[i, t+1])
    )
    for i in range(count_units) for t in range(count_times - 1)
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

In [13]:
# const - rampdown
# for t = 1, 2, ..., T-1
model.addConstrs(
    power_outputs[i, t-1] - power_outputs[i, t] <= (
        rampdowns[i] * commitments[i, t] 
        + startdowns[i] * (commitments[i, t-1] - commitments[i, t])
        +  power_outputs_maximum[i] * (1 - commitments[i, t-1])
    )
    for i in range(count_units) for t in range(1, count_times)
)
# t = 0
model.addConstrs(
    power_outputs_initial[i] - power_outputs[i, 0] <= (
        rampdowns[i] * commitments_initial[i] 
        + startdowns[i] * (commitments_initial[i] - commitments[i, 0])
        +  power_outputs_maximum[i] * (1 - commitments_initial[i])
    )
    for i in range(count_units)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [14]:
# const - minimum uptime main
# t = minimumups_initial[i] + 1 , ... , T - minimum_ups
model.addConstrs(
    gp.quicksum(commitments[i, tau] for tau in range(t, t+minimumups[i])) >= (
        minimumups[i] * (commitments[i, t] - commitments[i, t-1])
    )
    for i in range(count_units) for t in range(minimumups_initial[i] + 1, count_times - minimumups[i] + 1)
)
# t = minimumups_initial[i]
for i in range(count_units):
    t = minimumups_initial[i]
    lhs = gp.quicksum(commitments[i, tau] for tau in range(t, t + minimumups[i]))
    if t == 0:
        rhs = minimumups[i] * (commitments[i, 0] - commitments_initial[i])
    else:
        rhs = minimumups[i] * (commitments[i, t] - commitments[i, t-1])
    
    model.addConstr(lhs >= rhs)


In [15]:
# const - minimum uptime first hours
model.addConstrs(
    gp.quicksum(1 - commitments[i, t] for t in range(minimumups_initial[i])) == 0
    for i in range(count_units)
)
# const - minimum uptime last horus
model.addConstrs(
    gp.quicksum(
        commitments[i, tau] - (commitments[i, t] - commitments[i, t-1])
        for tau in range(t, count_times)
    ) >= 0
    for i in range(count_units) for t in range(count_times - minimumups[i] + 1, count_times)
)

{(0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>}

In [16]:
# const - minimum downtime main
# t = minimumdowns_initial[i] + 1, ..., T - minimumdowns (t \neq 0)
model.addConstrs(
    gp.quicksum(1 - commitments[i, tau] for tau in range(t, t + minimumdowns[i])) >= (
        minimumdowns[i] * (commitments[i, t-1] - commitments[i, t])
    )
    for i in range(count_units) for t in range(minimumdowns_initial[i] + 1, count_times - minimumdowns[i] + 1)
)
# t = minimumdowns_initial[i]
for i in range(count_units):
    t = minimumdowns_initial[i]
    lhs = gp.quicksum(1 - commitments[i, tau] for tau in range(t, t + minimumdowns[i]))
    if t == 0:
        rhs = minimumdowns[i] * (commitments_initial[i] - commitments[i, 0])
    else:
        rhs = minimumdowns[i] * (commitments[i, t-1] - commitments[i, t])

    model.addConstr(lhs >= rhs)

In [17]:
# const - minimum downtime first hours
model.addConstrs(
    gp.quicksum(commitments[i, t] for t in range(minimumdowns_initial[i])) == 0
    for i in range(count_units)
)

# const - minimum downtime last hours
model.addConstrs(
    gp.quicksum(
        1 - commitments[i, tau] - (commitments[i, t-1] - commitments[i, t])
        for tau in range(t, count_times)
    ) >= 0
    for i in range(count_units) for t in range(count_times - minimumdowns[i] + 1, count_times)
)

{(0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>}

In [ ]:
# generation cost
generation_cost = gp.quicksum(
    costs_generation[i] * power_outputs[i, t]
    for i in range(count_units) for t in range(count_times)
)

# startup cost
startup_cost = gp.quicksum(
    costs_startup[i, t]
    for i in range(count_units) for t in range(count_times)
)

# system cost
system_cost = generation_cost + startup_cost

# objective declaration
model.setObjective(system_cost, gp.GRB.MINIMIZE)

In [19]:
model.optimize()

In [20]:
model.ObjVal

8800.0

In [21]:
generation_cost.getValue()

8050.0

In [22]:
startup_cost.getValue()

750.0

In [32]:
xd = np.array([[commitments[i, t].X for t in range(count_times)] for i in range(count_units)])


In [34]:
xd

array([[ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1., -0.]])

In [38]:
xd[2, 0] <0

np.False_

In [39]:
xd.astype(np.uint8)

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=uint8)

In [26]:
model.getAttr("X", commitments)

{(0, 0): 1.0,
 (0, 1): 1.0,
 (0, 2): 1.0,
 (0, 3): 1.0,
 (0, 4): 1.0,
 (0, 5): 1.0,
 (1, 0): 1.0,
 (1, 1): 1.0,
 (1, 2): -0.0,
 (1, 3): -0.0,
 (1, 4): -0.0,
 (1, 5): -0.0,
 (2, 0): -0.0,
 (2, 1): -0.0,
 (2, 2): -0.0,
 (2, 3): -0.0,
 (2, 4): 1.0,
 (2, 5): -0.0}